## Demo to show how to read json file for training and building the dataset

In [11]:
import monai
from monai.data import load_decathlon_datalist, list_data_collate, CacheDataset, DataLoader
from monai.transforms import (
    Activations,
    EnsureChannelFirstd,
    AsDiscrete,
    Compose,
    LoadImaged,
    RandCropByPosNegLabeld,
    RandRotate90d,
    ScaleIntensityd,
)

In [1]:
datalist = "/var/data/student_home/lia/thesis/monai_segmentation/monai_training/JSON_dir/train.json"

In [3]:
train_files = load_decathlon_datalist(data_list_file_path=datalist, is_segmentation=True, data_list_key='training')

In [4]:
train_files

[{'fold': 0,
  'image': '/var/data/MONAI_Choroid_Plexus/dataset_train_from_scratch_monai/0_ChP.nii.gz',
  'label': '/var/data/MONAI_Choroid_Plexus/dataset_train_from_scratch_monai/labels/final/0_ChP.nii.gz'},
 {'fold': 0,
  'image': '/var/data/MONAI_Choroid_Plexus/dataset_train_from_scratch_monai/10_ChP.nii.gz',
  'label': '/var/data/MONAI_Choroid_Plexus/dataset_train_from_scratch_monai/labels/final/10_ChP.nii.gz'},
 {'fold': 0,
  'image': '/var/data/MONAI_Choroid_Plexus/dataset_train_from_scratch_monai/11_ChP.nii.gz',
  'label': '/var/data/MONAI_Choroid_Plexus/dataset_train_from_scratch_monai/labels/final/11_ChP.nii.gz'},
 {'fold': 0,
  'image': '/var/data/MONAI_Choroid_Plexus/dataset_train_from_scratch_monai/12_ChP.nii.gz',
  'label': '/var/data/MONAI_Choroid_Plexus/dataset_train_from_scratch_monai/labels/final/12_ChP.nii.gz'},
 {'fold': 0,
  'image': '/var/data/MONAI_Choroid_Plexus/dataset_train_from_scratch_monai/13_ChP.nii.gz',
  'label': '/var/data/MONAI_Choroid_Plexus/dataset_tr

In [13]:
# define transforms for image and segmentation

train_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        ScaleIntensityd(keys="image"),
        RandCropByPosNegLabeld(keys=["image", "label"], label_key="label", spatial_size=[180, 180, 180], pos=1, neg=1, num_samples=4
            ),
        RandRotate90d(keys=["image", "label"], prob=0.5, spatial_axes=[0, 2]),
    ]
)

# define dataset, data loader
check_ds = CacheDataset(data=train_files, transform=train_transforms)
# use batch_size=2 to load images and use RandCropByPosNegLabeld to generate 2 x 4 images for network training
check_loader = DataLoader(check_ds, batch_size=2, num_workers=4, collate_fn=list_data_collate)
check_data = monai.utils.misc.first(check_loader) # Returns the first item in the given iterable or default if empty, meaningful mostly with ‘for’ expressions.
print(check_data["image"].shape, check_data["label"].shape)

train_dataset = CacheDataset(
    data=train_files, transform=train_transforms, cache_num=6, cache_rate=1.0, num_workers=4)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=4, collate_fn=list_data_collate)



Loading dataset: 100%|██████████| 29/29 [00:23<00:00,  1.25it/s]


torch.Size([8, 1, 180, 180, 180]) torch.Size([8, 1, 180, 180, 180])


Loading dataset: 100%|██████████| 6/6 [00:02<00:00,  2.22it/s]


In [9]:
train_dataset

In [ ]:
python ASCHOPLEX/launching_tool.py --dataroot '/var/data/MONAI_Choroid_Plexus/dataset_aschoplex' --work_dir '/var/data/student_home/lia/ASCHOPLEX' --finetune 'yes' --prediction 'yes' 